In [59]:
import os
from os.path import join as opj
import numpy as np
import random

import open3d as o3d
from open3d.web_visualizer import draw
import matplotlib.pyplot as plt
import trimesh

from utils import *
from dataset import *

In [60]:
import pickle as pkl
data_root = 'data_root'
new_aff_path = opj(data_root, 'remapped_full_shape_train_data.pkl')
with open(new_aff_path, 'rb') as f:
    new_aff_dataset = pkl.load(f)
print("Loaded dataset with {} samples".format(len(new_aff_dataset)))


Loaded dataset with 1096 samples


In [61]:
# pick a random datapoint
# set_random_seed(0)
shape_id = random.choice(list(new_aff_dataset.keys()))
datapoint = new_aff_dataset[shape_id]
print("All affordances: ", datapoint["affordance_classes"])
print("Num All affordances: ", len(datapoint["affordance_classes"]))

All affordances:  ['handover', 'cut', 'stab', 'lift', 'wrap', 'pour', 'wear']
Num All affordances:  7


In [62]:
print("Shape id ", shape_id, "\nSemantic class: ", datapoint['semantic_class'])
pcl = datapoint['data']['coordinates']
# Viz pcl with o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcl)
# color black
pcd.paint_uniform_color([0, 0, 0])
# viz in jupyter nb
print("Drawing pcd with", len(pcl), "points")
draw(pcd)

Shape id  3d8444d8717cc0b7c18cdaa4851a3c95 
Semantic class:  Bottle
Drawing pcd with 2048 points
[Open3D INFO] Window window_30 created.


WebVisualizer(window_uid='window_30')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate


In [ ]:
# get pcls of all labels
labels_list = []
scores_list = []
for key in datapoint['data']['labels'].keys():
    labels = datapoint['data']['labels'][key]
    scores = labels.sum()
    print("Label: ", key, " Score: ", scores)
    labels_list.append(labels)
    scores_list.append(scores)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcl)
    pcd.paint_uniform_color([0, 0, 0])
    np.asarray(pcd.colors)[labels.squeeze() > 0] = np.random.uniform(0, 1, 3)
    print("Label 0: ", key)
    draw([pcd])

Label:  handover  Score:  222
Label 0:  handover
[Open3D INFO] Window window_31 created.


WebVisualizer(window_uid='window_31')

Label:  wrap  Score:  222
Label 0:  wrap
[Open3D INFO] Window window_32 created.


WebVisualizer(window_uid='window_32')

Label:  pour  Score:  317.87366
Label 0:  pour
[Open3D INFO] Window window_33 created.


WebVisualizer(window_uid='window_33')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.27025155726074446
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.27025155726074446
[Open3D INFO] Sending init frames to window_30.
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Ca

In [64]:
# Load and visualize the shapenet object

# Re-center and re-scale the point cloud

# Viz with o3d/trimesh